In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.16.1


In [2]:
train_file_path = "../SMS/train-data.tsv"
test_file_path = "../SMS/valid-data.tsv"

In [3]:
# Load and preprocess the data
def load_data(file_path):
    data = pd.read_csv(file_path, sep='\t', header=None, names=['label', 'text'])
    data['label'] = data['label'].map({'ham': 0, 'spam': 1})
    return data

train_data = load_data(train_file_path)
test_data = load_data(test_file_path)

# Tokenize the text data
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_data['text'])

train_sequences = tokenizer.texts_to_sequences(train_data['text'])
train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, padding='post', maxlen=50)

test_sequences = tokenizer.texts_to_sequences(test_data['text'])
test_padded = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, padding='post', maxlen=50)

train_labels = train_data['label'].values
test_labels = test_data['label'].values

In [4]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=16),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
# Train the model
history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels), verbose=2)

Epoch 1/10
131/131 - 3s - 26ms/step - accuracy: 0.8653 - loss: 0.3634 - val_accuracy: 0.8657 - val_loss: 0.3264
Epoch 2/10
131/131 - 1s - 5ms/step - accuracy: 0.8660 - loss: 0.3069 - val_accuracy: 0.8657 - val_loss: 0.2750
Epoch 3/10
131/131 - 1s - 4ms/step - accuracy: 0.9028 - loss: 0.2092 - val_accuracy: 0.9562 - val_loss: 0.1560
Epoch 4/10
131/131 - 1s - 4ms/step - accuracy: 0.9689 - loss: 0.1083 - val_accuracy: 0.9749 - val_loss: 0.0942
Epoch 5/10
131/131 - 1s - 4ms/step - accuracy: 0.9830 - loss: 0.0665 - val_accuracy: 0.9756 - val_loss: 0.0746
Epoch 6/10
131/131 - 1s - 4ms/step - accuracy: 0.9868 - loss: 0.0492 - val_accuracy: 0.9813 - val_loss: 0.0605
Epoch 7/10
131/131 - 1s - 5ms/step - accuracy: 0.9895 - loss: 0.0387 - val_accuracy: 0.9813 - val_loss: 0.0544
Epoch 8/10
131/131 - 1s - 4ms/step - accuracy: 0.9904 - loss: 0.0336 - val_accuracy: 0.9813 - val_loss: 0.0513
Epoch 9/10
131/131 - 1s - 4ms/step - accuracy: 0.9921 - loss: 0.0283 - val_accuracy: 0.9828 - val_loss: 0.0479


In [9]:
# Define the prediction function
def predict_message(pred_text):
    sequence = tokenizer.texts_to_sequences([pred_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, padding='post', maxlen=50)
    prediction = model.predict(padded_sequence)[0][0]
    label = 'spam' if prediction > 0.1 else 'ham'
    return [prediction, label]

# Test the function
pred_text = "urgent! call 09066350750 from your landline. your complimentary 4* ibiza holiday or 10,000 cash await collection sae t&cs po box 434 sk3 8wp 150 ppm 18+"
prediction = predict_message(pred_text)
print(prediction)

pred_text = "message important information for o2 user. today is your lucky day! 2 find out why log onto http://www.urawinner.com there is a fantastic surprise awaiting you."
prediction = predict_message(pred_text)
print(prediction)

pred_text = "i dont want to go. can we try it a different day? available sat"
prediction = predict_message(pred_text)
print(prediction)

pred_text = "our new mobile video service is live. just install on your phone to start watching."
prediction = predict_message(pred_text)
print(prediction)

pred_text = "shit that is really shocking and scary, cant imagine for a second. def up for night out. do u think there is somewhere i could crash for night, save on taxi?"
prediction = predict_message(pred_text)
print(prediction)

pred_text = "i'll bring it tomorrow. don't forget the milk."
prediction = predict_message(pred_text)
print(prediction)

pred_text = "Congratulations! You have won a $1000 cash prize. Call now to claim."
prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
[0.99872726, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[0.9944251, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[0.0008536595, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[0.84301764, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
[0.006363359, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[0.0004636744, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
[0.9600493, 'spam']


In [7]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
You passed the challenge. Great job!
